In [1]:
### 다른 클러스터링 사용해보기

In [1]:
import pandas as pd
import numpy as np

# Data Check
- Session 테이블의 TOT_PAG_VIEW_CT와 TOT_SESS_HR_V가 Nan인 경우 비정상적인 접근(ex. 접속하자마자 종료된 경우)이라고 생각하여 삭제하였음

In [3]:
prod = pd.read_csv("01.Pruduct.csv")
sear = pd.read_csv("02.Search1.csv")
sess = pd.read_csv("05.Session.csv")
mast = pd.read_csv("06.Master.csv")
#print(prod.shape)
#print(sear.shape)
print(sess.shape)
#print(mast.shape)

C:\Users\User\anaconda3.7\envs\chaelin_virtenv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2712907, 9)


In [4]:
print(prod.isnull().sum())
print(sear.isnull().sum())
print(sess.isnull().sum())
print(mast.isnull().sum())

CLNT_ID      0
SESS_ID      0
HITS_SEQ     0
PD_C         0
PD_ADD_NM    0
PD_BRA_NM    0
PD_BUY_AM    0
PD_BUY_CT    0
dtype: int64
CLNT_ID       0
SESS_ID       0
KWD_NM        0
SEARCH_CNT    0
dtype: int64
CLNT_ID                0
SESS_ID                0
SESS_SEQ               0
SESS_DT                0
TOT_PAG_VIEW_CT      274
TOT_SESS_HR_V      14202
DVC_CTG_NM             0
ZON_NM                 0
CITY_NM                0
dtype: int64
PD_C        0
PD_NM       0
CLAC1_NM    0
CLAC2_NM    0
CLAC3_NM    0
dtype: int64


In [5]:
sess[sess['TOT_SESS_HR_V'].isnull()].head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
38,5878756,7465325,5,20180529,1.0,NaN,mobile,Chungcheongnam-do,Cheonan-si
64,6065738,9564520,26,20180424,1.0,NaN,mobile,Gyeonggi-do,Gwangmyeong-si
65,6065738,9564529,25,20180424,1.0,NaN,mobile,Gyeonggi-do,Gwangmyeong-si
246,6003295,1035552,2,20180913,1.0,NaN,desktop,Daejeon,Daejeon
604,5946142,9600252,4,20180423,1.0,NaN,mobile,Seoul,Seoul


In [6]:
sess[sess['TOT_PAG_VIEW_CT'].isnull()].head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
11409,5075856,6836578,19,20180608,NaN,NaN,mobile,Seoul,Seoul
26628,5368135,4391880,10,20180718,NaN,NaN,mobile,Chungcheongnam-do,Yeongi-gun
43241,5687186,852560,40,20180916,NaN,NaN,mobile,Seoul,Seoul
58494,6104032,1568619,78,20180904,NaN,NaN,mobile,Chungcheongnam-do,Cheonan-si
64700,3735745,6884635,1,20180607,NaN,NaN,desktop,Gwangju,Gwangju


In [7]:
sess = sess.dropna()
print(sess.isnull().sum())
print(sess.shape)

CLNT_ID            0
SESS_ID            0
SESS_SEQ           0
SESS_DT            0
TOT_PAG_VIEW_CT    0
TOT_SESS_HR_V      0
DVC_CTG_NM         0
ZON_NM             0
CITY_NM            0
dtype: int64
(2698696, 9)


In [8]:
sess['TOT_SESS_HR_V'] = sess['TOT_SESS_HR_V'].map(lambda x: x.replace(",", ""))
sess.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
0,5873599,8641867,9,20180509,82.0,1890,mobile,Daejeon,Daejeon
1,5873599,6616320,21,20180611,105.0,1604,mobile,Busan,Busan
2,5873599,5886172,40,20180624,41.0,632,mobile,Daejeon,Daejeon
3,5873884,1050889,15,20180913,160.0,1035,mobile,Gyeonggi-do,Anyang
4,5874461,10298270,5,20180412,13.0,298,mobile,Seoul,Seoul


# Clustering
- 세션 일련 번호, 총 페이지 조회 건수, 총 세션 시간 값, 기기 유형, 행동을 기준으로 클러스터링

In [9]:
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
%matplotlib inline

In [10]:
act_buy = prod[['SESS_ID']]
act_buy['ACTION'] = 'buy'

act_sear = sear[['SESS_ID']]
act_sear['ACTION'] = 'search'

act = pd.concat([act_buy, act_sear], axis=0, sort=False)
print(act['ACTION'].unique())
act = act.drop_duplicates()
act = act.groupby('SESS_ID').agg(lambda x: x.tolist())

C:\Users\User\anaconda3.7\envs\chaelin_virtenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\User\anaconda3.7\envs\chaelin_virtenv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


['buy' 'search']


In [11]:
#act['ACTION'].unique

act['ACTION'] = act['ACTION'].map(lambda x: "".join(x) if len(x)<2 else ",".join(x))
print(act['ACTION'].unique())         #.unique 는 유일한 값 찾는것.. 

['buy' 'buy,search']


In [12]:
print(act.tail())
df = pd.merge(sess, act, left_on='SESS_ID', right_on='SESS_ID', how='left')
df.shape

              ACTION
SESS_ID             
10977908         buy
10977924         buy
10977932  buy,search
10977936  buy,search
10977941         buy


(2698696, 10)

In [13]:
df = df[['CLNT_ID', 'SESS_ID','SESS_SEQ','TOT_PAG_VIEW_CT','TOT_SESS_HR_V','DVC_CTG_NM','ACTION']]
df.head()

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION
0,5873599,8641867,9,82.0,1890,mobile,"buy,search"
1,5873599,6616320,21,105.0,1604,mobile,"buy,search"
2,5873599,5886172,40,41.0,632,mobile,"buy,search"
3,5873884,1050889,15,160.0,1035,mobile,"buy,search"
4,5874461,10298270,5,13.0,298,mobile,buy


In [15]:
from sklearn.preprocessing import LabelEncoder

encode_DVC = LabelEncoder()
encode_DVC.fit(df['DVC_CTG_NM'])

df['DVC_CTG_NM'] = encode_DVC.transform(df['DVC_CTG_NM'])

encode_ACT = LabelEncoder()
encode_ACT.fit(df['ACTION'])

df['ACTION'] = encode_ACT.transform(df['ACTION'])

print(set(df['DVC_CTG_NM']))
print(set(df['ACTION']))
df.head()

{0, 1, 2}
{0, 1}


,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION
0,5873599,8641867,9,82.0,1890,1,1
1,5873599,6616320,21,105.0,1604,1,1
2,5873599,5886172,40,41.0,632,1,1
3,5873884,1050889,15,160.0,1035,1,1
4,5874461,10298270,5,13.0,298,1,0


### 최적의 클러스터 갯수

In [ ]:
cols = ['SESS_SEQ','TOT_PAG_VIEW_CT','TOT_SESS_HR_V','DVC_CTG_NM','ACTION']
for num in range(3,7):
    print("For n_clusters = {}".format(num))
    clusterer = MiniBatchKMeans(n_clusters=num)
    preds = clusterer.fit_predict(df[cols])
    score = silhouette_score(df[cols], preds)
    print("silhouette score is {}".format(score))

For n_clusters = 3


In [20]:
y_pred_outliers = clf.predict(data)

out = pd.DataFrame(y_pred_outliers)
out = out.rename(columns={0:"out"})
out_data = pd.concat([data, out],1)

In [26]:
print(out_data.shape)
print(data.shape)  #  와이라노와이라노~

(6423888, 9)
(4064969, 8)


In [ ]:
이게 원래 코드!! cl_df 데이터를 갖고 최적의 클러스터 갯수 구하는 중
#cols = ['SESS_SEQ','TOT_PAG_VIEW_CT','TOT_SESS_HR_V','DVC_CTG_NM','ACTION']
#for num in range(3,7):
#    print("For n_clusters = {}".format(num))
#    clusterer = MiniBatchKMeans(n_clusters=num)
#    preds = clusterer.fit_predict(cl_df[cols])
#    score = silhouette_score(cl_df[cols], preds)
#    print("silhouette score is {}".format(score))

In [ ]:
#cl_df['CLUSTER'] = preds

In [ ]:
#sns.pairplot(cl_df, hue="CLUSTER", diag_kws={'bw': 0.2})
#plt.show()

In [ ]:
#item_set = cl_df[['CLNT_ID', 'CLUSTER']].groupby('CLNT_ID').agg(lambda x:x.tolist())

# Frequent Pattern Mining

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
#frequent_itemsets = apriori(itemset, min_support=0.5, use_colnames=True)

In [ ]:
#association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Itemset For Each Pattern